# Overview

This notebook combines all the necessary predictors for modelling into one ImageCollection (using the Google Earth Engine (GEE) framework). Plots are provided throughout to aid with understanding. These bands of predictors are then sampled at the field data points required to make a tabular dataset of predictor values at each field data location. This tabular dataset is output and is what is used to develop the machine learning models in the next notebook.

Note: to use this notebook a GEE account is required. Any raster (.tif) datasets that are required for the notebook, must be uploaded to the GEE assets page and the paths updated within the notebook below. Vector datasets (.shp) do not need to be uploaded.

## Predictor List

Currently implemented preditors:
1. Sentinel-2 with cloud removal (2 options): all bands and 3 indices 
2. SAR (Sentinel-1 and PALSAR, 2 bands of imagery and 3 indices each)
3. Distance to rivers and watercourses
4. Landcover and crop maps
5. Elevation and derived indices 

Experimental predictors:
1. NATMAP predictors (vector, soilscapes, carbon)
2. Peaty soils map

# Setup

In [1]:
# necessary imports 
import ee
import geemap
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, mean_squared_error

# if error with geopandas then try running:
# pip install --upgrade --force-reinstall shapely

/Users/hamishcampbell/opt/miniconda3/envs/geospatial/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# data and file paths (update as necessary)
path_depths = "/Users/hamishcampbell/Documents/Cambridge/PhD/1st Year/Code/depth-mapping/Data/Ground Truth/depth_samples_historical.shp" 
path_fenOutline = "/Users/hamishcampbell/Documents/Cambridge/PhD/1st Year/Code/depth-mapping/Data/CLR/NCA_46_The_Fens.shp"
path_vector = "projects/ee-depthmapping-hrac2/assets/Rasterised_NATMAPvector"
path_soilscapes = "projects/ee-depthmapping-hrac2/assets/Rasterised_NATMAPsoilscapes"
path_carbon = "projects/ee-depthmapping-hrac2/assets/Rasterised_NATMAPcarbon"
path_rivers = "projects/ee-depthmapping-hrac2/assets/distance_to_rivers"
path_watercourses = "projects/ee-depthmapping-hrac2/assets/distance_to_watercourses"
path_landcover = "projects/ee-depthmapping-hrac2/assets/landcover"
path_crome = "projects/ee-depthmapping-hrac2/assets/CROME"
path_peatySoils = "projects/ee-depthmapping-hrac2/assets/peaty_soils"
path_elevation = "projects/ee-depthmapping-hrac2/assets/Lidar_DTM_10m"
path_MRRTF = "projects/ee-depthmapping-hrac2/assets/MRRTF"
path_MRVBF = "projects/ee-depthmapping-hrac2/assets/MRVBF"
path_TWI = "projects/ee-depthmapping-hrac2/assets/TWI"


In [3]:
# initialise Google Earth Engine

# NOTE: first run only or after some time has elapsed
#ee.Authenticate()

ee.Initialize()

In [4]:
# load the Fens outline shape file into GEE
fenOutline = ee.FeatureCollection(geemap.shp_to_ee(path_fenOutline))

# load the depth measurements shape file into GEE
depths = geemap.shp_to_ee(path_depths)

# Predictors

## Sentinel-2

### Cloud removal option 1 - "Mean Composite"

This cloud removal method finds all the non-cloudy pixels in the area of interest, within the time interval specified, and calculates the mean value of each pixel. The "CLD_PRB_THRESH" variable is used to specify what percentage of cloud is allowable before a pixel is labelled as cloudy (100 means nothing is labelled cloudy and so all pixels are used for mean calculation). The pixel cloudiness values are given by 's2cloudless' which is a seperate algorithm whose result can be obtained through GEE (provides a cloud probability value of 0-100).  

In [5]:
# cloud masking paramters to control aggression - values chosen to find best balance between cloud-removal and few
# NA data points
AOI = fenOutline
START_DATE = '2022-04-01'
END_DATE = '2022-07-01'
CLOUD_FILTER = 60                 # tiles with more than this percentage are removed from analysis
CLD_PRB_THRESH = 40               # pixels greater than this are considered clouds 
NIR_DRK_THRESH = 0.15             # used for cloud shadow detection
CLD_PRJ_DIST = 1
BUFFER = 50 

In [6]:
# Function to attach an s2cloudless tile to each of the sentinel tiles that denotes which pixels are likely to be clouds
def attach_cld_msk_to_sen(aoi, start_date, end_date):
    # Import and filter Sentinel-2 imagery
    s2_images = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless pixel values
    s2_cloudless = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the Sentinel collection by the 'system:index' (time) property
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_images,
        'secondary': s2_cloudless,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

In [7]:
# get Sentinel tiles with s2cloudless tiles attached
sentinel_with_s2cloudless = attach_cld_msk_to_sen(AOI, START_DATE, END_DATE)

In [8]:
# Function to add band with cloud mask information to Sentinel imagery
def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))


# Function to add band with cloud shadow mask information to Sentinel imagery
def add_shadow_bands(img):
    # Identify water pixels from the SCL band
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels)
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection)
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))


# Function to execute the addition of cloud and shadow bands to imagery and add combined mask
def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)


# Function to actually mask the images using the mask bands to remove clouds/shadows
def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)

In [9]:
# Apply all of the cloud/shadow mask functions to mask Sentinel imagery, and then calculate the mean composite image 
# from all the non-cloudy pixels at each point (mask determines which values get used in mean calculation)
sen_algorithm1 = (sentinel_with_s2cloudless.map(add_cld_shdw_mask)
                                           .map(apply_cld_shdw_mask)
                                           .mean())

### Cloud removal option 2 - "Prioritise Contiguous Tiles"

In [10]:
# get all surface reflectance Sentinel-2 imagery 
sen = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")

# select a start and end date for imagery composite to be made up of 
start_date = '2022-06-01'
end_date = '2022-07-01'

# keep only the data that meets some basic criteria
sen_filtered = (sen.filterBounds(fenOutline)
    .filterDate(start_date, end_date)               # temporally limited 
    .sort('CLOUD_COVERAGE_ASSESSMENT', False))      # sort tiles by cloud cover 

In [11]:
# Write a function that alters the pixel validity mask of a tile according to if cloud is present or not
def check_cloud_absence(tile): 
    # if MSK_CLDPRB less than 10 => low chance of cloud => let mask remain valid at that pixel
    # if medium-high chance of cloud (>10) => update mask to say pixel is invalid (mask = 0)
    return tile.updateMask(tile.select('MSK_CLDPRB').lte(10))

In [12]:
# obtain a (mostly) cloud-free mosaic for each quarter 
quarterly_mosaics = []
filter_start = ee.Date('2022-01-01')

for quarter in range(0,4):
    
    # keep only tiles from the quarter being studied
    quarterly_sen = sen_filtered.filterDate(filter_start, filter_start.advance(3,'month'))
    
    # Update the pixel validity mask to include condition on cloud absence
    quarterly_sen = quarterly_sen.map(check_cloud_absence)
        
    # stitch the most cloud-free tiles together and replace remaining cloud pixels with those from next tile with presence
    # NOTE: by sorting on cloud presence we keep complete image as temporally contiguous as possible
    quarterly_mosaic = quarterly_sen.mosaic()
    
    # store the results required
    quarterly_mosaics.append(quarterly_mosaic)
    
    # repeat for next quarter
    filter_start = filter_start.advance(3,'month')
    

In [13]:
# select the quarter of the year to use for modelling (1/2/3/4)
quarter = 2
sen_algorithm2 = quarterly_mosaics[quarter-1]

### Selection and Plotting

In [14]:
# select which of the cloud algorithms we want to use
sen = sen_algorithm1

In [15]:
# select all the useful Sentinel bands and rename them for interpretability
sen = sen.select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12'],
                  ['Blue', 'Green', 'Red', 'REdge1', 'REdge2', 'REdge3', 'NIR', 'REdge4', 'SWIR1', 'SWIR2'])

In [16]:
# plot the result (method 1 takes quite a while to render - zoom in and out to speed up)
Map = geemap.Map()
visParams = {
  'bands': ['Red', 'Green', 'Blue'],
  'max': 3000,
  'min': 0,
}
Map.centerObject(fenOutline, 8)
Map.addLayer(sen.clip(fenOutline), visParams, 'Sentinel 2')
Map

# plot the Fens outline in red
style = {
    'color': 'red',
    'fillColor': '00000000'
}
Map.centerObject(fenOutline, 8)
Map.addLayer(fenOutline.style(**style), {}, 'Fen Outline')
Map

Map(center=[52.70465287691509, 0.062270855934294995], controls=(WidgetControl(options=['position', 'transparen…

### Sentinel Indices 

In [17]:
# calculate NDVI, used to highlight vegetation presence or greenness
ndvi = sen.normalizedDifference(['NIR', 'Red']).rename('NDVI')

# calculate NDWI, used to highlight waterbodies 
ndwi = sen.normalizedDifference(['Green', 'NIR']).rename('NDWI')

# calculate NDMI, used to highlight water content in plants
ndmi = sen.normalizedDifference(['NIR', 'SWIR1']).rename('NDMI')

In [18]:
Map = geemap.Map()
visParams = {
  'max': 1,
  'min': -1,
}
Map.centerObject(fenOutline, 8)
Map.addLayer(ndvi.clip(fenOutline), visParams, 'NDVI')

style = {
    'color': 'red',
    'fillColor': '00000000'
}
Map.centerObject(fenOutline, 8)
Map.addLayer(fenOutline.style(**style), {}, 'Fen Outline')
Map

Map(center=[52.70465287691509, 0.062270855934294995], controls=(WidgetControl(options=['position', 'transparen…

In [19]:
# add the indices to the other sentinel-2 predictors 
sen = sen.addBands([ndvi, ndwi, ndmi])

## SAR x2

Since our experience of using SAR data is virtually non-existent, we follow Rudiyanto's steps heavily. We use both Sentinel-1 imagery (c-band) and also PALSAR (L-band)

### Sentinel-1

In [20]:
# load the SAR data from the GEE catalog
sen1 = ee.ImageCollection('COPERNICUS/S1_GRD').filterDate(START_DATE, END_DATE)

# Rudiyanto paper species to use polarisations of: VV and VH
sen1 = (sen1.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')))

# to get consistent images filter to take only ascending images (Fens is flat-ish)
sen1 = sen1.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))

# take mean of VV and VH polarisations and clip to Fens
sen1_VV = sen1.select('VV').mean().clip(fenOutline).rename("Sen1_VV")
sen1_VH = sen1.select('VH').mean().clip(fenOutline).rename("Sen1_VH")

In [21]:
# calculate indices per Rudiyanto paper:
# index1 = VV/VH
sen1_index1 = sen1_VV.divide(sen1_VH).rename("Sen1_Index1")

# index2 = ((VV+VH)/2)
sen1_index2 = sen1_VV.add(sen1_VH).divide(2).rename("Sen1_Index2")

# index3 = VV-VH
sen1_index3 = sen1_VV.subtract(sen1_VH).rename("Sen1_Index3")

In [22]:
# put all the relevant predictors in one image stack
sen1_stack = sen1_VV.addBands([sen1_VH, sen1_index1, sen1_index2, sen1_index3])
sen1_stack

In [23]:
# plot some results
Map = geemap.Map()
visParams = {
    'min': -20,
    'max': 0,
}
Map.addLayer(sen1_VV, visParams, 'Sen1')
Map.centerObject(fenOutline, 8)

style = {
    'color': 'red',
    'fillColor': '00000000'
}
Map.addLayer(fenOutline.style(**style), {}, 'Fen Outline')
Map

Map(center=[52.70465287691509, 0.062270855934294995], controls=(WidgetControl(options=['position', 'transparen…

### PALSAR

In [24]:
# load the palsar data (one annual image per year, so take most recent =2020)
palsar = ee.ImageCollection("JAXA/ALOS/PALSAR/YEARLY/SAR").filter(ee.Filter.date('2020-01-01', '2021-01-01'))

# select HH and HV polarisation as per Rudiyanto, and clip to Fens
palsar_HH = palsar.select('HH').first().clip(fenOutline).rename("PALSAR_HH")
palsar_HV = palsar.select('HV').first().clip(fenOutline).rename("PALSAR_HV")

In [25]:
# calculate indices per Rudiyanto paper:
# index1 = HH/HV
palsar1_index1 = palsar_HH.divide(palsar_HV).rename("PALSAR_Index1")

# index2 = ((HH+HV)/2)
palsar1_index2 = palsar_HH.add(palsar_HV).divide(2).rename("PALSAR_Index2")

# index3 = HH-HV
palsar1_index3 = palsar_HH.subtract(palsar_HV).rename("PALSAR_Index3")

In [26]:
# put all the relevant predictors in one image stack
palsar_stack = palsar_HH.addBands([palsar_HV, palsar1_index1, palsar1_index2, palsar1_index3])
palsar_stack

In [27]:
# plot some results
Map = geemap.Map()
visParams = {
  'min': 0,
  'max': 10000,
}
Map.addLayer(palsar_HH, visParams, 'PALSAR');

style = {
    'color': 'red',
    'fillColor': '00000000'
}
Map.centerObject(fenOutline, 8)
Map.addLayer(fenOutline.style(**style), {}, 'Fen Outline')
Map

Map(center=[52.70465287691509, 0.062270855934294995], controls=(WidgetControl(options=['position', 'transparen…

## Distance to Watercourses

We have two datsets: distance to major rivers and distance to all watercourses (e.g. ditches, streams etc)

In [28]:
# load the raster image from the GEE assets location 
d2_rivers = ee.Image(path_rivers)
d2_rivers = d2_rivers.rename(['d2_River'])

d2_watercourses = ee.Image(path_watercourses)
d2_watercourses = d2_watercourses.rename(['d2_Watercourse'])

# combine the predictors as bands 
d2_water = d2_rivers.addBands(d2_watercourses)
d2_water = d2_water.clip(fenOutline)

In [29]:
# plot this distance data on a map
Map = geemap.Map()
visParams = {
    'min': 0,
    'max': 7000,
}
Map.addLayer(d2_water.select('d2_River'), visParams, 'Distance to River')
Map.centerObject(fenOutline, 8)
Map

Map(center=[52.70465287691509, 0.062270855934294995], controls=(WidgetControl(options=['position', 'transparen…

## Landcover Datasets

In [30]:
# load the raster image from the GEE assets location 
landcover = ee.Image(path_landcover)
landcover = landcover.rename(['Landcover', 'Landcover_conf'])

crome = ee.Image(path_crome)
crome = crome.rename(['CROPTYPE'])

# combine the predictors as bands 
landcover_datasets = landcover.addBands(crome)

In [31]:
# plot this categorical crop data on a new map
Map = geemap.Map()
visParams = {
    'min': 1,
    'max': 35,
}
Map.addLayer(landcover_datasets.select('CROPTYPE'), visParams, 'Landcover')
Map.centerObject(fenOutline, 8)
Map

Map(center=[52.70465287691509, 0.062270855934294995], controls=(WidgetControl(options=['position', 'transparen…

## Elevation

In [32]:
# load the raster images from the GEE assets location 
elevation = ee.Image(path_elevation)
elevation = elevation.rename(['Elevation'])

MRRTF = ee.Image(path_MRRTF)
MRRTF = MRRTF.rename(['MRRTF'])

MRVBF = ee.Image(path_MRVBF)
MRVBF = MRVBF.rename(['MRVBF'])

TWI = ee.Image(path_TWI)
TWI = TWI.rename(['TWI'])

# combine the predictors as bands 
elevation_datasets = elevation.addBands([MRRTF, MRVBF, TWI])

In [33]:
# plot this lidar data on a map
Map = geemap.Map()
visParams = {
    'min': -10,
    'max': 30,
}
Map.addLayer(elevation_datasets.select('Elevation'), visParams, 'Lidar')
Map.centerObject(fenOutline, 8)
Map

Map(center=[52.70465287691509, 0.062270855934294995], controls=(WidgetControl(options=['position', 'transparen…

## NATMAP Predictors

### NATMAP Vector

In [34]:
# load the raster image from the GEE assets location (cannot load local raster files unlike with .shp files)
NATMAP_vector = ee.Image(path_vector)
NATMAP_vector = NATMAP_vector.rename(['Soiltype'])
NATMAP_vector = NATMAP_vector.cast({'Soiltype': 'uint16'})

In [35]:
# plot this categorical soil data on a new map
Map = geemap.Map()
visParams = {
    'min': 0,
    'max': 76,
}
Map.addLayer(NATMAP_vector.select('Soiltype'), visParams, 'Soil Map')
Map.centerObject(fenOutline, 8)
Map

Map(center=[52.70465287691509, 0.062270855934294995], controls=(WidgetControl(options=['position', 'transparen…

### NATMAP Soilscapes

In [36]:
# load the raster image 
NATMAP_soilscapes = ee.Image(path_soilscapes)
NATMAP_soilscapes = NATMAP_soilscapes.rename(['Soilscapes'])

In [37]:
# plot the soilscape data
Map = geemap.Map()
visParams = {
    'min': 0,
    'max': 20,
}
Map.addLayer(NATMAP_soilscapes, visParams, 'Soilscapes Map')
Map.centerObject(fenOutline, 8)
Map

Map(center=[52.70465287691509, 0.062270855934294995], controls=(WidgetControl(options=['position', 'transparen…

### NATMAP Carbon

In [38]:
# load the raster image (with 3 bands - 0 to 30, 30 to 100 and 100 to 150 carbon stock)
NATMAP_carbon = ee.Image(path_carbon)
NATMAP_carbon = NATMAP_carbon.rename(['AV_STK_150', 'AV_STK_30', 'AV_STK_100'])
NATMAP_carbon = NATMAP_carbon.clip(fenOutline)

In [39]:
# plot this carbon stock data 
Map = geemap.Map()
visParams = {
    'min': 0,
    'max': 86,
}
Map.addLayer(NATMAP_carbon.select('AV_STK_100'), visParams, 'Carbon Map')
Map.centerObject(fenOutline, 8)
Map

Map(center=[52.70465287691509, 0.062270855934294995], controls=(WidgetControl(options=['position', 'transparen…

### Combining NATMAP Predictors

In [40]:
NATMAP_preds = NATMAP_vector.addBands([NATMAP_soilscapes, NATMAP_carbon])

## Peaty Soils

Note that the peaty soils map is quite similar to the LandIS soil maps as they are derived from similar data sources

In [41]:
# load the raster images from the GEE assets location 
peaty_soils = ee.Image(path_peatySoils)
peaty_soils = peaty_soils.rename(['Peaty Soils'])

# wherever there is no data we want a value of 0
peaty_soils = peaty_soils.unmask(0)

In [42]:
# plot this peaty soils data on a map
peaty_soils = peaty_soils.clip(fenOutline)

Map = geemap.Map()
visParams = {
    'min': 0,
    'max': 2,
}
Map.addLayer(peaty_soils, visParams, 'Peaty Soils')
Map.centerObject(fenOutline, 8)
Map

Map(center=[52.70465287691509, 0.062270855934294995], controls=(WidgetControl(options=['position', 'transparen…

## Combining Predictors

We need to make a general data layer stack (one image with bands corresponding to predictors)

In [43]:
# start with Sentinel-2 data
predictor_stack = sen

# add the NATMAP LandIS soil map datasets
predictor_stack = predictor_stack.addBands(NATMAP_preds)

# add the SAR predictors 
predictor_stack = predictor_stack.addBands(sen1_stack)
predictor_stack = predictor_stack.addBands(palsar_stack)

# add distance to water predictors 
predictor_stack = predictor_stack.addBands(d2_water) 

# add the landcover datasets as predictors 
predictor_stack = predictor_stack.addBands(landcover_datasets) 

# add the lidar elevation data and derived indices
predictor_stack = predictor_stack.addBands(elevation_datasets) 

# add the peaty soils map
predictor_stack = predictor_stack.addBands(peaty_soils) 

# double check we clip to the Fens region
predictor_stack = predictor_stack.clip(fenOutline) 

We have decided that it is best if all predictors have a 10m resolution. This reprojection only works for making datasets smaller resolution. This command applies to all bands (predictors).

In [44]:
# make sure all the predictors are using a spatial resolution of 10m (this crs uses metres and is for UK area)
predictor_stack = predictor_stack.reproject('EPSG:3040', scale=10) 

In [45]:
# print the result
predictor_stack

# Sampling at Field Data Locations

We need a feature collection containing the predictor data for each of the training depth locations. This means we need to:
1. Sample the bands at the ground-truth points
2. Inner join the GT data and predictor data using their geometries
3. Save the resulting tabular dataset

## 1. Sampling

In [46]:
# sample the predictor stack data at each of the GT points
pred_samples = predictor_stack.sample(depths.geometry(), scale=10, geometries=True) 

## 2. Inner Join (and buffering)

In [47]:
# define an algorithm to be applied to each predictor sample: we want to buffer the geometry assigned to each sample
# by 10m, since the current geometry is approximate i.e. does not match the GT points used to guide sampler exactly
def bufferPoints(currentFeature, pastFeatureList):
    return ee.List(pastFeatureList).add(currentFeature.buffer(10))

# apply the algorithm to each sample in our collection and store the result in a list
pred_samples = pred_samples.iterate(bufferPoints, ee.List([]))

# convert this list back into a FeatureCollection
pred_samples = ee.FeatureCollection(pred_samples.getInfo())

In [49]:
# Setup the inner join operation between the predictors and GT datasets

# make a filter object that assesses intersections of objects. '.geo' specifies that for the 1st (left) and 2nd (right)
# objects being considered we want to apply the filter to the geometry fields of each feature instead of a property
intersectFilter = ee.Filter.intersects(leftField='.geo', rightField='.geo')

# define an inner join operation
IJ_operation = ee.Join.inner('predictor_data', 'depth_data')

# apply the join
combinedDS = IJ_operation.apply(pred_samples, depths, intersectFilter)


In [50]:
# The result has a complex nested structure that isn't useful for the operations we want. Thus, re-define a new dataset
# with the same information but an improved structure 
def extract_depths(feature): 
    return feature.get('depth_data')

def extract_predictors(feature): 
    return feature.get('predictor_data')

# get information associated with the depth subcollection
depth_collection = combinedDS.map(extract_depths)

# get information associated with the predictor subcollection
predictor_collection = combinedDS.map(extract_predictors)


In [52]:
# get the relevant information from the depth dataset subcollection
geometries = depth_collection.geometry().getInfo()['coordinates']
locations = depth_collection.aggregate_array('Location').getInfo()
partners = depth_collection.aggregate_array('Partner').getInfo()
thicknesses = depth_collection.aggregate_array('Thickness').getInfo()

# the date (year) we need for the model requires a bit of extra formatting
dates = depth_collection.aggregate_array('Date').getInfo()
for idx, date in enumerate(dates):
    dates[idx] = int(date[-4:])

# now deal with the predictor collection
predictor_names = predictor_collection.first().propertyNames().getInfo()
predictor_names.remove('system:index')

# make a list of lists of each predictor's values
predictor_values = []
for bandName in predictor_names:
    predictor_values.append(predictor_collection.aggregate_array(bandName).getInfo())

In [53]:
# make a new feature collection that stores this information efficiently
data = []
for sample_idx, coords in enumerate(geometries):
    
    # get the location, partner, depth and date of the sample of interest
    sample_data = {'location': locations[sample_idx],
                   'partner': partners[sample_idx],
                   'depth': thicknesses[sample_idx],
                   'date': dates[sample_idx]}
    
    # get the predictor values at this sample
    for pred_idx, predictor_name in enumerate(predictor_names):
        sample_data[predictor_name] = predictor_values[pred_idx][sample_idx]
        
    # store the geometry and values of the sample
    feature = ee.Feature(ee.Geometry.Point(coords), sample_data)
    data.append(feature)
    
# finally convert the data into a feature collection    
dataset = ee.FeatureCollection(data)

In [54]:
# save the predictor/label dataset locally as a csv
geemap.ee_to_csv(dataset, filename='depth_dataset.csv')

Generating URL ...
Please wait ...
Data downloaded to /Users/hamishcampbell/Documents/Cambridge/PhD/1st Year/Code/depth-mapping/notebooks/depth_dataset.csv
